# Initializing chunking

In [1]:
import a
file_name='input/md/hindi/Natural_Farming_Kharif_Booklet.md'
md_file=open(file_name,'r',encoding='utf-8')
md=md_file.read()
chunks=md.split('\n')

# chunks=[chunk for chunk in chunks if chunk!='']
# for chunk in chunks:
#     print(chunk,end='\n*****\n')

ModuleNotFoundError: No module named 'a'

In [ ]:
chunks

In [ ]:
def table_parser(chunks,index):
    inside_table = False
    current_table = ''
    for i,chunk in enumerate(chunks):
        if inside_table:
            # Check if the current chunk is the end of the tablere
            if chunk.startswith('|') :
                inside_table = True
                current_table=current_table+'\n'+chunk
            else:

                return current_table,(index+i)
        elif chunk.startswith('|'):
            inside_table = True
            current_table=chunk
    #     elif chunk=='':
    #         continue
#         else:
#             print('post',(index+i))
#             return combined_chunks,(index+i)
    return 0


def para_parser(chunks,index):
    inside_para=False
    current_para=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if chunk=='':
            i=i+1            
        elif inside_para:
            if chunk[0] not in temp:
                inside_para=True
                current_para=current_para+' '+chunk
            else:
                current_para=current_para.strip()
                # if len(current_para)==0:
                #     return(current_para,'flag')
                return (current_para,(index+i))            
        elif (chunk[0] not in temp) or chunk!='|':
            inside_para=True
            current_para=chunk
            
    return (current_para,(index+i))

def list_parser(chunks,index):
    inside_list=False
    current_list=''
    temp=['#','## ','### ','#### ','##### ','###### ','-','|']
    for i,chunk in enumerate(chunks):
        if inside_list:
            if chunk=='' or chunk.startswith('- ') or chunk.startswith('* '):
                return (current_list,(index+i))
            if chunk[0] not in temp:
                current_list=current_list+'\n'+chunk
        elif chunk.startswith('-') or chunk.startswith('* '):
            current_list=chunk
            inside_list=True

    return (current_list,(index+i))

# Extracting types for chunks and creating a JSON

In [ ]:
# for i in range(len(chunks)):
i=0
combined_chunks=[]
temp=['#','## ','### ','#### ','##### ','###### ','-','*','|']
while i<len(chunks):
    if chunks[i]=='':
        combined_chunks.append({'type':'return','content':chunks[i]})
        i=i+1
    elif chunks[i].startswith('|') and len(chunks[i])!=1:
        chunk,i=table_parser(chunks[i:],i)
        combined_chunks.append({'type':'table','content':chunk})

    elif chunks[i].startswith('# ') or chunks[i].startswith('## ') or chunks[i].startswith('### ') or chunks[i].startswith('#### ') or chunks[i].startswith('##### '):
        combined_chunks.append({'type':'heading','content':chunks[i]})
        i=i+1

    elif chunks[i].startswith('- ') or chunks[i].startswith('* '):
        chunk,i=para_parser(chunks[i:],i)
        combined_chunks.append({'type':'list','content':chunk})

    elif (chunks[i][0] not in temp and (i+1)!=len(chunks)) or chunks[i]=='|':
        chunk,i=para_parser(chunks[i:],i)
            
            # chunk=chunks[i].strip()
        if len(chunk)==0:
                
                continue
        combined_chunks.append({'type':'para','content':chunk})
            # i=i+1
    else:
        i=i+1

In [ ]:
combined_chunks = [chunk for chunk in combined_chunks if chunk['type'] != 'return']
# combined_chunks

In [ ]:
combined_chunks

In [ ]:
inside_pair=False
paired_chunks=[]
current_pair={'type':'heading-para pair','content':{}}
for chunk in combined_chunks:
    if chunk['type']=='heading':
        inside_pair=True
        current_pair['content']['heading']=chunk['content']
    elif chunk['type']=='para':
        if inside_pair:
            current_pair['content']['para']=chunk['content']
            paired_chunks.append(current_pair)
            inside_pair=False
            current_pair={'type':'heading-para pair','content':{}}
        else:
            paired_chunks.append(chunk)
    else:
        paired_chunks.append(chunk)

In [ ]:
# def count_level(content):
#     level
#     if content.startswith('# '):
#         level=1
#     elif content.startswith('## '):
#         level=2
#     elif content.startswith('### '):
#         level=3
#     elif content.startswith('#### '):
#         level=4
#     elif content.startswith('##### '):
#         level=5
#     elif content.startswith('###### '):
#         level=6


#     return level

# Creating heading heirarchy

In [ ]:
def assign_ids_and_parents(chunks):
    id_counter = {'1': 0, '2': 0, '3': 0,'4':0,'5':0}
    parent_stack = []

    for chunk in chunks:
        if chunk['type'] == 'heading':
            level = chunk['content'].count('#')
#             level=count_level(chunk['content'])
#             print('hehe',level)
            id_counter[str(level)] += 1
            chunk['id'] = f'{level}{id_counter[str(level)]}'

            if parent_stack==[] or level==1:
                parent_stack = [chunk['id']]
            else:
                buff=int(str(parent_stack[0])[0])-1
                while len(parent_stack)>=(level-buff):
                    parent_stack.pop()
                parent_stack.append(chunk['id'])
            chunk['parents'] = parent_stack[:-1]
#             print(parent_stack)

        else:
            chunk['parents'] = parent_stack.copy() if parent_stack else []

    return chunks
final_chunks=assign_ids_and_parents(combined_chunks)
final_chunks

In [ ]:
len(final_chunks)

# Creating CSV

In [ ]:
headings={}
for chunk in final_chunks:
    if chunk['type']=='heading':
        headings[chunk['id']]=chunk['content']

In [ ]:
def get_subheading(chunks,index):
  subheadings=''
  for heading in chunks[index]['parents']:
    subheadings+=(' '.join(headings[heading].split()[1:]))+','
  return subheadings[:-1]

import csv
#CSV columns: chunkId,content,StartPage,EndPage,heading,ContentWordCount,pdfName,contentType,image,summary
def export_csv(final_chunks):

  with open('output.csv','w') as file:
    csv_writer = csv.writer(file)
    data=['chunkId','content','StartPage','EndPage','heading','ContentWordCount','pdfName','contentType','image','summary']
    csv_writer.writerow(data)

  for i,chunk in enumerate(final_chunks):
    chunkId=i+1
    content=chunk['content']
    heading=get_subheading(final_chunks,i)
    contentType=chunk['type']
    ContentWordCount=len(content.split()) if contentType!='table' else content.split('|')
    pdfName=file_name.split('.')[0]+'.pdf'
    data=[chunkId,content,'','',heading,ContentWordCount,pdfName,contentType,'','']
    with open('output.csv','a',encoding='utf-8') as file:
      csv_writer = csv.writer(file)
      csv_writer.writerow(data)




In [ ]:
original_string = "This is a simple string."

# Split the string into words, add a space after each word, and join them
modified_string = ' '.join(original_string.split()[1:])

# Display the modified string
print(modified_string)

In [ ]:
export_csv(final_chunks)

# Translating

In [ ]:
import requests
import json
import pandas as pd
import time

def translate_text_bhashini(input_text):
    url = "https://dhruva-api.bhashini.gov.in/services/inference/pipeline"

    payload = json.dumps({
        "pipelineTasks": [
            {
                "taskType": "translation",
                "config": {
                    "language": {
                        "sourceLanguage": "hi",
                        "targetLanguage": "en"
                    },
                    "serviceId": "ai4bharat/indictrans-v2-all-gpu--t4"
                }
            }
        ],
        "inputData": {
            "input": [
                {
                    "source": input_text
                }
            ]
        }
    })
    headers = {
        'Accept': '*/*',
        'User-Agent': 'Thunder Client (https://www.thunderclient.com)',
        'Authorization': 'sLAFJehUCZQ72NIM4nDZNCya7TQVzittLgJEU0vIf-69rp0gFUcGu5sjwAaOSUfa',
        'Content-Type': 'application/json'
    }

    retries = 0
    max_retries = 5
    start_time = time.time()
    while retries < max_retries:
        try:
            response = requests.post(url, headers=headers, data=payload)
            # Check if the request was successful
            if response.status_code == 200:
                translated_output = json.loads(response.text)['pipelineResponse'][0]['output'][0]['target']
                end_time = time.time()
                return translated_output, retries, end_time - start_time
            else:
                print(f"Request failed with status code {response.status_code}. Retrying...")
                retries += 1
        except Exception as e:
            print(f"An error occurred: {e}. Retrying...")
            retries += 1

    end_time = time.time()
    return "Translation failed after maximum retries.", max_retries, end_time - start_time

In [ ]:
a

In [ ]:
import os
input_folder='output\\hindi'
all_files=[]
for folder, subfolders, files in os.walk(input_folder):
    full_file_paths = [os.path.join(folder, file) for file in files]
    all_files.extend(full_file_paths)
all_files

In [ ]:
import pandas as pd
df=pd.read_csv(all_files[0])
newdf=pd.read_csv('output.csv')

In [ ]:
newdf

In [ ]:
df = df.rename(columns={'content': 'originalLanguageChunk','heading':'originalLanguageHeading'})

In [ ]:
print('translating')
df['content'] = df.apply(lambda row: translate_text_bhashini(row['originalLanguageChunk'])[0] if row['contentType'] != 'table' else row['originalLanguageChunk'], axis=1)
df['heading'] = df.apply(lambda row: translate_text_bhashini(row['originalLanguageHeading'])[0] if row['contentType'] != 'table' else row['originalLanguageHeading'], axis=1)
print('\rtranslating done')
df=df[]

In [ ]:
df=df[['chunkId','content','heading','originalLanguageChunk','StartPage','EndPage','originalLanguageHeading','ContentWordCount','pdfName','contentType','image','summary']]

In [ ]:
df

# Splitting bigger chunks

In [ ]:
import re
import pandas as pd

In [ ]:
df=pd.read_csv('output/english\\FARMER PRODUCER ORGANISATIONS.csv')

In [ ]:
new_rows = []
count=0
for index, row in df.iterrows():
    
    
    word_count = row['ContentWordCount']
    content = row['content']
    
    # Check if ContentWordCount is greater than 200
    if int(word_count) > 200 and row['contentType']=='para':
        # Split the content into chunks of <= 200 words with complete sentences
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', content)
        
        
        chunks = []
        current_chunk = []
        for sentence in sentences:
            # print(sentence)
            
            temp=' '.join(current_chunk + [sentence])
            if len(temp.split()) <= 200:
                current_chunk.append(sentence)
                # print(current_chunk)
                
            else:
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
        
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        # Create new rows for each chunk
        
        for chunk in chunks:
            count+=1
            new_row = {
                'chunkId': index,  # You may need to assign a new chunkId
                'content': chunk,
                'heading': row['heading'],
                'originalLanguageChunk': row['originalLanguageChunk'],
                'StartPage': row['StartPage'],
                'EndPage': row['EndPage'],
                'originalLanguageHeading': row['originalLanguageHeading'],
                'ContentWordCount': len(chunk.split()),
                'pdfName': row['pdfName'],
                'contentType': row['contentType'],
                'image': row['image'],
                'summary': row['summary']
            }
            new_rows.append(new_row)
    else:
        # If ContentWordCount is <= 200, keep the row as it is
        count+=1
        new_rows.append({
            'chunkId': count,
            'content': content,
            'heading': row['heading'],
            'originalLanguageChunk': row['originalLanguageChunk'],
            'StartPage': row['StartPage'],
            'EndPage': row['EndPage'],            
            'originalLanguageHeading': row['originalLanguageHeading'],
            'ContentWordCount': word_count,
            'pdfName': row['pdfName'],
            'contentType': row['contentType'],
            'image': row['image'],
            'summary': row['summary']
        })

# Create a new DataFrame with the updated rows
new_df = pd.DataFrame(new_rows)


In [ ]:
new_df

In [ ]:
new_d

In [ ]:
sentence="and Rural Development National Bank for Agriculture Mumbai  2015 Title                               :                                  Farmer Producer Organisations - Frequently Asked Questions (FAQs)   Written and Published by                               :                                  Farm Sector Policy Department & Farm Sector                                   Development Department, NABARD Head Office, Mumbai  Date of Publishing                               :                                  March 2015 Design & Printing                               :                                  M/s Image Impression Contact                               :                                  Plot No C-24, 'G' Block, Bandra Kurla Complex,                                  Bandra East, Mumbai - 400051."

In [ ]:
a=' '.join(current_chunk + [sentence])

In [ ]:
len(a)

# 2.0

In [ ]:
import pandas as pd
import re
def split_chunks(df,upper_limit=180):
    new_rows = []
    count=0
    for index, row in df.iterrows():
        
    
        word_count = row['ContentWordCount']
        content = row['content']
        
        if int(word_count) > upper_limit and row['contentType']=='para':
            sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', content)
            
            
            chunks = []
            current_chunk = []
            for sentence in sentences:
                # print(sentence)
                
                temp=' '.join(current_chunk + [sentence])
                if len(temp.split()) <= upper_limit:
                    current_chunk.append(sentence)
                    # print(current_chunk)
                    
                else:
                    chunks.append(' '.join(current_chunk))
                    current_chunk = [sentence]
            
            if current_chunk:
                chunks.append(' '.join(current_chunk))
    
            
            
            for chunk in chunks:
                count+=1
                new_row = {
                    'chunkId': index,  
                    'content': chunk,
                    'heading': row['heading'],
                    'originalLanguageChunk': row['originalLanguageChunk'],
                    'StartPage': row['StartPage'],
                    'EndPage': row['EndPage'],
                    'originalLanguageHeading': row['originalLanguageHeading'],
                    'ContentWordCount': len(chunk.split()),
                    'pdfName': row['pdfName'],
                    'contentType': row['contentType'],
                    'image': row['image'],
                    'summary': row['summary']
                }
                new_rows.append(new_row)
        else:
           
            count+=1
            new_rows.append({
                    'chunkId': count,
                    'content': content,
                    'heading': row['heading'],
                    'originalLanguageChunk': row['originalLanguageChunk'],
                    'StartPage': row['StartPage'],
                    'EndPage': row['EndPage'],            
                    'originalLanguageHeading': row['originalLanguageHeading'],
                    'ContentWordCount': word_count,
                    'pdfName': row['pdfName'],
                    'contentType': row['contentType'],
                    'image': row['image'],
                    'summary': row['summary']
            })


    new_df = pd.DataFrame(new_rows)
    return new_df

In [ ]:
df=pd.read_csv('output/english\\NFSM12102018.csv')

In [ ]:
new_df=split_chunks(df)

In [ ]:
new_df

# Joining Rows

In [115]:
import pandas as pd
df=pd.read_csv('output/english/Revised_Operational_Guidelines_new.csv')
df['status']='visited'

In [116]:
rows = df.to_dict(orient='records')

In [119]:
print('%')

%


In [118]:
rows = df.to_dict(orient='records')
size=len(rows)
i=0
lowerLimit=40
while i<size:
    
    row=rows[i]
    if row['status']=='unvisited':
        i+=1
        continue
    # print(row['chunkId'])
    print(i)
    prev_row=rows[i-1] if i>0 else None
    next_row=rows[i+1] if i<size-1 else None
    
    # if type(str(next_row))=='pandas.core.series.Series':

    # merged_rows = []
    # current_row = None
    temp=1
    while row['ContentWordCount']<lowerLimit:
        print('here')
        
        if isinstance(next_row, dict):
            
            same_type=(row['contentType']==next_row['contentType'])
            same_heading=row['heading']==next_row['heading']
            valid_word_count=row['ContentWordCount']+next_row['ContentWordCount']<200 
        else:
            same_type=same_heading=valid_word_count=False            
        if isinstance(prev_row, dict):
            psame_type=(row['contentType']==prev_row['contentType'])
            psame_heading=(row['heading']==prev_row['heading'])
            pvalid_word_count=row['ContentWordCount']+prev_row['ContentWordCount']<200
        else:
            psame_type=psame_heading=pvalid_word_count=False
       
        if  same_type and same_heading and valid_word_count:
            print('here')
            row['content']+=next_row['content']
            row['ContentWordCount']+=next_row['ContentWordCount']
            next_row['status']='unvisited'
            
        # elif same_type and not same_heading:
        elif psame_type and psame_heading and pvalid_word_count:
            row['content']+=prev_row['content']
            row['ContentWordCount']+=prev_row['ContentWordCount']
            prev_row['status']='unvisited'

        elif valid_word_count and same_heading:
            row['content']+=next_row['content']
            row['ContentWordCount']+=next_row['ContentWordCount']
            next_row['status']='unvisited'

        elif pvalid_word_count and psame_heading:
            row['content']+=prev_row['content']
            row['ContentWordCount']+=prev_row['ContentWordCount']
            prev_row['status']='unvisited'

        elif valid_word_count:
            row['content']+=(next_row['content'])
            row['ContentWordCount']+=next_row['ContentWordCount']
            next_row['status']='unvisited'
            merged=str(row['heading'])+','+str(next_row['heading'])
            merged=list(set(merged.split(',')))
            
            row['heading']=','.join(merged)
        
        elif pvalid_word_count:
            row['content']+=prev_row['content']
            row['ContentWordCount']+=prev_row['ContentWordCount']
            prev_row['status']='unvisited'
            merged=str(row['heading'])+','+str(prev_row['heading'])
            merged=list(set(merged.split(',')))
            # row['heading']=','.join(list(set([(row['heading']+','+prev_row['heading'])].split(','))))

        else:
            print('reached')
            temp+=1
            
            prev_row=rows[i-temp] if i>temp-1 else None
            # print(prev_row)
            # break
            next_row=rows[i+temp] if i<size-temp else None
            # print(prev_row)
            
    # break
            # temp+=1
            # if i>temp-1:
            #     prev_row=rows[i-temp]
            # else:
            #     prev_row=None
            # next_row=rows[i+temp] if i<size-temp else None

    i+=1

0
here
reached
here
reached
here
reached
here


TypeError: can only concatenate str (not "float") to str

In [109]:
columns = ['Unnamed: 0', 'chunkId', 'content', 'heading', 'originalLanguageChunk', 'StartPage', 'EndPage', 'originalLanguageHeading', 'ContentWordCount', 'pdfName', 'contentType', 'image', 'summary', 'status']

# Create DataFrame
new_df = pd.DataFrame(rows, columns=columns)
new_df = new_df[new_df['status']=='visited']
new_df=new_df.drop(columns=['Unnamed: 0','status'])

In [110]:
new_df=new_df.drop(columns=['Unnamed: 0','status'])

KeyError: "['Unnamed: 0', 'status'] not found in axis"

In [84]:
new_df.to_csv('output/english/FPO_Scheme_Guidelines_FINAL_English_splitted_merged.csv',index=False)

# Summary

In [ ]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key=os.environ.get("API_KEY")

def get_gpt_response(system_prompt,user_prompt):
  response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-1106", temperature = 0,
                messages=[{"role": "system", "content": system_prompt},
                          {"role": "user", "content": user_prompt}
                ])
  gpt_response =  response["choices"][0]["message"]["content"]
  return(gpt_response)

In [ ]:

system_prompt = """You are an AI bot that summarises user provided content in 5 sentences.
The content that you are shared is part of the Samagra Processes and Policy Corpus. The data was extracted from the PDF and has two sections - Heading and Content.
These will be provided to you to summarize in the folloowing format

Heading: <Heading>
Content: <Content>

You dont say anything else apart from the 5 sentence summary. Do not reproduce either the Heading or Content.
"""
file='E:\PDF-Parser\output\english\FARMER PRODUCER ORGANISATIONS.csv'
df = pd.read_csv(file)

df['summary'] = ''
for i in range(df.shape[0]):
    print(i)
    if df.loc[i,'contentType']!='heading':
        content = "Heading:" + str(df.loc[i,'heading']) + "\n" + "Content: " + str(df.loc[i,'content'])
    
        df.loc[i,'summary'] =  get_gpt_response(system_prompt,content)
df.to_csv(file)

In [ ]:
# summary('E:\PDF-Parser\output\english\FARMER PRODUCER ORGANISATIONS.csv')
df.loc[0,'contentType']

In [ ]:
df

# DUMP

In [ ]:
combined_chunks = []
inside_table = False
current_table = ''

for chunk in chunks:
    if inside_table:
        # Check if the current chunk is the end of the table
        if chunk.startswith('|'):
            inside_table = True
            current_table=current_table+'\n'+chunk
        else:
            combined_chunks.append(current_table)
            combined_chunks.append(chunk)
            inside_table=False
    elif chunk.startswith('|'):
        inside_table = True
        current_table=chunk
#     elif chunk=='':
#         continue
    else:
        combined_chunks.append(chunk)

In [ ]:
inside_pair=False
current_pair=''
for chunk in combined_chunks:
    if chunk['type']=='heading':
        iside_pair=True
    elif chunk['type']=='para':
        if inside_pair

In [ ]:
for i in combined_chunks:
    print(i,end='\n************\n')

## GPT Stuff (No use)

In [ ]:
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup

In [ ]:
file='Value_chain_financing.md'
file=open(file,'r',encoding='utf-8')
md_text=file.read()

In [ ]:
from markdown_it import MarkdownIt

def md_to_html(md_file_path, html_file_path):
    # Read Markdown content from file
    with open(md_file_path, 'r', encoding='utf-8') as md_file:
        md_text = md_file.read()
    md = MarkdownIt()
    html_content = md.render(md_text)
    with open(html_file_path, 'w', encoding='utf-8') as html_file:
        html_file.write(html_content)

md_path = "Value_chain_financing.md"
html_path = "output.html"
md_to_html(md_path, html_path)


In [ ]:
file=open('output.html', 'r', encoding='utf-8')
html_content = file.read()
page_data = {}
current_main_heading = ''
current_subheading = ''
crop_soup = BeautifulSoup(html_content, 'html.parser')
# print(crop_soup)
tags = crop_soup.find_all(['h4','h1','h2','h3', 'p', 'li','table'])

for tag in tags:
    if tag.name == 'h4':
        current_main_heading = tag.text.strip()
        current_subheading = ''  # Reset subheading when a new main heading is found
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
    elif tag.name == 'p' and tag.find('strong'):
        current_subheading = tag.text.strip()
        if current_main_heading not in page_data:
            page_data[current_main_heading] = {}
        if current_subheading not in page_data[current_main_heading]:
            page_data[current_main_heading][current_subheading] = []
    elif current_main_heading and current_subheading:
        # Only add content if both main heading and subheading are available
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


    else:
        if current_main_heading and not current_subheading:
            current_subheading='General'
            page_data[current_main_heading][current_subheading]=[]
        if not current_main_heading and not current_subheading:
            current_main_heading='General'
            current_subheading='General'
            page_data[current_main_heading]={}
            page_data[current_main_heading][current_subheading]=[]
        if tag.name=='table':
            data=table_parser(tag)
        else:
            data=tag.text.strip()
        page_data[current_main_heading][current_subheading].append(data)


In [ ]:
page_data

In [ ]:
md = MarkdownIt()
tokens = md.parse(md_text)

headers = []
current_header = None

for token in tokens:
    print(token,end='\n\n\n')